In [440]:
import json
import requests
import hmac
import hashlib
import time
import pandas as pd
import cbpro
from datetime import datetime, timedelta
from joblib import load

In [399]:
with open('../coinbase-secrets.json', 'r') as jfile:
    secrets = json.loads(jfile.read())
    jfile.close()

## Coinbase

In [400]:
# Create custom authentication for Coinbase API
class CoinbaseWalletAuth(requests.auth.AuthBase):
    def __init__(self, api_key, secret_key):
        self.api_key = api_key
        self.secret_key = secret_key

    def __call__(self, request):
        timestamp = str(int(time.time()))
        message = timestamp + request.method + request.path_url + (request.body or '')
        signature = hmac.new(
            self.secret_key.encode('utf-8'), 
            message.encode('utf-8'), 
            hashlib.sha256
        ).hexdigest()

        request.headers.update({
            'CB-ACCESS-SIGN': signature,
            'CB-ACCESS-TIMESTAMP': timestamp,
            'CB-ACCESS-KEY': self.api_key,
            'CB-VERSION': '2021-06-25'
        })
        return request


In [401]:
api_url = 'https://api.coinbase.com/v2'
api_url = 'https://api-public.sandbox.pro.coinbase.com'
auth = CoinbaseWalletAuth(secrets['API_KEY'], secrets['API_SECRET'])

In [402]:
r = requests.get(f'{api_url}/prices/ETH-USD/buy', auth=auth)
r.json()

{'message': 'Invalid API Key'}

In [403]:
r = requests.get(f'{api_url}/prices/ETH-USD/sell', auth=auth)
r.json()

{'message': 'Invalid API Key'}

In [404]:
r = requests.get(f'{api_url}/prices/ETH-USD/spot', auth=auth)
r.json()

{'message': 'Invalid API Key'}

## Coinbase Pro

In [437]:
public_client = cbpro.PublicClient()

In [407]:
public_client.get_product_ticker(product_id='ETH-USD')

{'trade_id': 134271293,
 'price': '2179.4',
 'size': '0.22482524',
 'time': '2021-06-29T12:33:19.869372Z',
 'bid': '2179.39',
 'ask': '2179.4',
 'volume': '252522.79356988'}

In [408]:
time_data = public_client.get_time()
end = '2021-06-28T23:59:59.999Z'
start = '2021-06-28T23:00:00.000Z'

In [409]:
columns = ['timestamp', 'low', 'high', 'open', 'close', 'volume']

In [410]:
historic_data = public_client.get_product_historic_rates(
    product_id = 'ETH-USD', 
    start=str(start), 
    end=str(end), 
    granularity=3600
)
historic_data = pd.DataFrame(historic_data, columns=columns)
historic_data['timestamp'] = pd.to_datetime(historic_data['timestamp'], unit='s')
historic_data

,timestamp,low,high,open,close,volume
0,2021-06-28 23:00:00,2062.35,2097.5,2097.5,2083.79,9784.226516


In [413]:
end = time_data['iso']
print(end)

2021-06-29T12:33:22.078Z


In [435]:
end_datetime = datetime.fromtimestamp(time_data['epoch'])
start_datetime = end_datetime - timedelta(hours=1)
end_iso = end_datetime.isoformat()
start_iso = start_datetime.isoformat()

In [438]:
historic_data = public_client.get_product_historic_rates(
    product_id = 'ETH-USD', 
    start=start_iso, 
    end=end_iso, 
    granularity=3600
)
historic_data = pd.DataFrame(historic_data, columns=columns)
historic_data['timestamp'] = pd.to_datetime(historic_data['timestamp'], unit='s')
historic_data

,timestamp,low,high,open,close,volume
0,2021-06-29 08:00:00,2115.33,2156.18,2131.82,2148.73,6156.64763


In [449]:
model = load('../models/linear_model.joblib')
batch = historic_data[['open', 'high', 'low', 'close', 'volume']].values
prediction = model.predict(batch)
prediction

array([2148.79605238])

In [ ]:
class AssetTrader(object):
    
    def __init__(self, asset: str, api_secret: str, use_sandbox: bool = True):
        self.asset = asset
        self.api_secret = api_secret
        self.public_client = cbpro.PublicClient()
        api_url = ''
        if use_sandbox:
            api_url = "https://api-public.sandbox.pro.coinbase.com"
        
        self.private_client = cbpro.AuthenticatedClient(
            key, 
            b64secret, 
            passphrase,
            api_url=api_url
        )
            
    def _get_start_end_iso_times(self, current_timestamp: str):
        """
        From the current iso formatted timestamp, this generates
        a start and end datetime that are 1 hour apart.
        
        :param current_timestamp: (str) ISO-8601 Formatted Datetimestamp.
        :returns: (tuple) Contains (start, end) datetimes.
        """
        public_client = self.
        time_data = self.public_client.get_time()
        
        end_datetime = datetime.fromtimestamp(time_data['epoch'])
        start_datetime = end_datetime - timedelta(hours=1)
        
        end_iso = end_datetime.isoformat()
        start_iso = start_datetime.isoformat()
        return (start_iso, end_iso)

        
    def get_asset_details_last_hour(self, start: str, end: str, granularity: int = 3600):
        """
        Retrieves hourly open, high, low, close, and volume for the given asset
        over the time range on start to end broken into granularity seconds.
        
        :param start: (str) ISO-8601 formatted timestamp.
        :param end: (str) ISO-8601 formatted timestamp.
        :param granularity: (int) Number of seconds per interval between start and end.
        :returns: (np.array) Array containing the detailed asset price data.
        """
        historic_data = public_client.get_product_historic_rates(
            product_id = self.asset, 
            start=start, 
            end=end, 
            granularity=granularity
        )
        historic_data = pd.DataFrame(historic_data, columns=columns)
        historic_data['timestamp'] = pd.to_datetime(historic_data['timestamp'], unit='s')
        historic_data = historic_data[['open', 'high', 'low', 'close', 'volume']]
        
        return historic_data.values